In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("data/data_2400.csv")
df.head()

In [ ]:
df.columns

In [ ]:
df["horse_name"].value_counts()

## 指定した馬のnレース前までのデータを列に追加する

In [ ]:
test_df = df[df["horse_name"] == "ステイブラビッシモ"].loc[:, :"horse_ranking_index"]
test_df

## 日時情報を整形
日時情報を使ってデータフレームをソートしたデータから、nレース前までのデータを取得するために行う。

In [ ]:
def add_date_data(df, year_column_name, month_column_name,  day_column_name):
    df["date"] = df.apply(lambda row: f"{row[year_column_name]}-{row[month_column_name]}-{row[day_column_name]}", axis=1)
    df["date"] = pd.to_datetime(df["date"])

In [ ]:
add_date_data(test_df, "race_year", "race_month", "race_day")
test_df.info()

## nレース前までのデータを列に追加する

In [ ]:
def create_horse_data(n_race, df):
    df.sort_values("date", inplace=True)
    race_df = df.iloc[-1:, :].reset_index(drop=True)
    columns = race_df.columns
    for n in range(1, n_race+1):
        add_columns = list(map(lambda c: c+f"_{n}", columns))
        add_race = df.iloc[-1-n:-n, :].reset_index(drop=True)
        add_race.columns = add_columns
        race_df = pd.concat([race_df, add_race], axis=1)
    return race_df

In [ ]:
# 3レース前までのデータを列に追加
result = create_horse_data(3, test_df)
result

In [ ]:
def extract_previous_race(n, df):
    return df.loc[:, f"race_year_{n}": f"horse_ranking_index_{n}"]

In [ ]:
extract_previous_race(1, result)

In [ ]:
extract_previous_race(2, result)

In [ ]:
extract_previous_race(3, result)